In [16]:
// Imports
import scala.sys.process._
import java.net.URL
import java.io.File
import java.io.File
import java.nio.file.{Files, StandardCopyOption}
import java.net.HttpURLConnection 
import org.apache.spark.sql.functions._

//import scala.sqlContext.implicits._
import org.apache.spark.sql.functions.udf
import org.apache.spark.sql.{Row, SparkSession}
import org.apache.spark.sql.functions.{col, to_date, to_timestamp}
import org.apache.spark.sql.types.DateType
import org.apache.spark.sql.functions.{from_unixtime, unix_timestamp, _}
import org.apache.spark.sql.types.{IntegerType, StringType, StructType}
import org.apache.spark.sql.SaveMode

import org.apache.spark.input.PortableDataStream
import java.util.zip.ZipInputStream
import java.io.BufferedReader
import java.io.InputStreamReader
import org.apache.spark.sql.SQLContext
import com.amazonaws.services.s3.AmazonS3Client
import com.amazonaws.auth.BasicAWSCredentials

//import org.apache.spark.sql.cassandra._

//import com.datastax.spark.connector._
import org.apache.spark.sql.types.IntegerType


import com.amazonaws.services.s3.{AmazonS3, AmazonS3ClientBuilder}


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import scala.sys.process._
import java.net.URL
import java.io.File
import java.io.File
import java.nio.file.{Files, StandardCopyOption}
import java.net.HttpURLConnection
import org.apache.spark.sql.functions._
import org.apache.spark.sql.functions.udf
import org.apache.spark.sql.{Row, SparkSession}
import org.apache.spark.sql.functions.{col, to_date, to_timestamp}
import org.apache.spark.sql.types.DateType
import org.apache.spark.sql.functions.{from_unixtime, unix_timestamp, _}
import org.apache.spark.sql.types.{IntegerType, StringType, StructType}
import org.apache.spark.sql.SaveMode
import org.apache.spark.input.PortableDataStream
import java.util.zip.ZipInputStream
import java.io.BufferedReader
import java.io.InputStreamReader
import org.apache.spark.sql.SQLContext
import com.amazonaws.services.s3.AmazonS3Client
import com.amazonaws.auth.BasicAWSCredentials
import org.apache.spark.sql.types.IntegerType
import com.amazonaws.services.s3.{AmazonS3, AmazonS3ClientBuilder}


In [2]:
def fileDownloader(urlOfFileToDownload: String, fileName: String) = {
    val url = new URL(urlOfFileToDownload)
    val connection = url.openConnection().asInstanceOf[HttpURLConnection]
    connection.setConnectTimeout(5000)
    connection.setReadTimeout(5000)
    connection.connect()

    if (connection.getResponseCode >= 400)
        println("error")
    else
        url #> new File(fileName) !!
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

fileDownloader: (urlOfFileToDownload: String, fileName: String)Any


In [3]:
def StokeFileS3(URL: String) = {
    val fileName = URL.split("/").last
    val dir = "/mnt/tmp/"
    val localFileName = dir + fileName
    try {
            fileDownloader(URL,  localFileName)
            @transient val s3Client: AmazonS3 = AmazonS3ClientBuilder.defaultClient()
            val localFile = new File(localFileName)
            s3Client.putObject("testfuret/Master_file ", fileName, localFile)   
            localFile.delete()
        
    } catch {
        case e: java.io.FileNotFoundException => 
        case e: com.amazonaws.SdkClientException => 

    }    
    
    
}



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

StokeFileS3: (URL: String)AnyVal


# Decompressage de fichier

### Pour Master 

In [4]:
// English Data
val list_csv = spark.read.format("csv").option("delimiter", " ").
    csv("s3://testfuret/masterfilelist.txt").
    withColumnRenamed("_c0","size").
    withColumnRenamed("_c1","hash").
    withColumnRenamed("_c2","url")

val list_2020_tot = list_csv.where(col("url").like("%/2020%")) 
//val list_2020_export_tot = list_2020_tot.where(col("url").like("%mention%")) 
list_2020_tot.show(10,false)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

list_csv: org.apache.spark.sql.DataFrame = [size: string, hash: string ... 1 more field]
list_2020_tot: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [size: string, hash: string ... 1 more field]
+-------+--------------------------------+--------------------------------------------------------------------+
|size   |hash                            |url                                                                 |
+-------+--------------------------------+--------------------------------------------------------------------+
|162032 |f7a7e782ee56c9eee521b232e1f8a52e|http://data.gdeltproject.org/gdeltv2/20200101000000.export.CSV.zip  |
|131781 |b68d3a524f3238bdd48ea4fa10976ae8|http://data.gdeltproject.org/gdeltv2/20200101000000.mentions.CSV.zip|
|5288568|cfa2331d641aca029fea28bebb462ed4|http://data.gdeltproject.org/gdeltv2/20200101000000.gkg.csv.zip     |
|142044 |dea3910f2cebdb760c2b7078d56eaed4|http://data.gdeltproject.org/gdeltv2/20200101001500.export.CSV.zip  |
|137798 |

In [5]:
val gkgRDD = sc.binaryFiles("s3://testfuret/Master_file /*gkg*",100).    
   flatMap {  
       
       // decompresser les fichiers
       case (name: String, content: PortableDataStream) =>

          val zis = new ZipInputStream(content.open)
          Stream.continually(zis.getNextEntry).
                takeWhile(_ != null).
                flatMap { _ =>
                    val br = new BufferedReader(new InputStreamReader(zis))
                    Stream.continually(br.readLine()).takeWhile(_ != null)
                }

    }

val gkgDF = gkgRDD.map(x => x.split("\t")).toDF()//.map(row => row.mkString(";")).map(x => x.split(";")).toDF()



val dfGKG = gkgDF.withColumn("_tmp", $"value").select(
    $"_tmp".getItem(0).as("GKGRECORDID"),
    $"_tmp".getItem(1).as("V2DATE"),
    //$"_tmp".getItem(2).as("V2SOURCECOLLECTIONIDENTIFIER"),
    $"_tmp".getItem(3).as("V2SOURCECOMMONNAME"),
    $"_tmp".getItem(4).as("V2DOCUMENTIDENTIFIER"),
    //$"_tmp".getItem(5).as("V1COUNTS"),
    //$"_tmp".getItem(6).as("V2.1COUNTS"),
    $"_tmp".getItem(7).as("V1THEMES"),
    //$"_tmp".getItem(8).as("V2ENHANCEDTHEMES"),
    $"_tmp".getItem(9).as("V1LOCATIONS"),
    //$"_tmp".getItem(10).as("V2ENHANCEDLOCATIONS"),
    $"_tmp".getItem(11).as("V1PERSONS"),
    //$"_tmp".getItem(12).as("V2ENHANCEDPERSONS"),
    //$"_tmp".getItem(13).as("V1ORGANIZATIONS"),
    //$"_tmp".getItem(14).as("V2ENHANCEDORGANIZATIONS"),
    $"_tmp".getItem(15).as("V1TONE")
    //$"_tmp".getItem(16).as("V2.1ENHANCEDDATES"),
    //$"_tmp".getItem(17).as("V2GCAM"),
    //$"_tmp".getItem(18).as("V21SHARINGIMAGE"),
    //$"_tmp".getItem(19).as("V21RELATEDIMAGES"),
    //$"_tmp".getItem(20).as("V21SOCIALIMAGEEMBEDS"),
    //$"_tmp".getItem(21).as("V21SOCIALVIDEOEMBEDS"),
    //$"_tmp".getItem(22).as("V21QUOTATIONS"),
    //$"_tmp".getItem(23).as("V21ALLNAMES"),
    //$"_tmp".getItem(24).as("V21AMOUNTS"),
    //$"_tmp".getItem(25).as("V21TRANSLATIONINFO"),
    //$"_tmp".getItem(26).as("V2EXTRASXML")
    )
dfGKG.show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

gkgRDD: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[15] at flatMap at <console>:56
gkgDF: org.apache.spark.sql.DataFrame = [value: array<string>]
dfGKG: org.apache.spark.sql.DataFrame = [GKGRECORDID: string, V2DATE: string ... 6 more fields]
+-----------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|      GKGRECORDID|        V2DATE|  V2SOURCECOMMONNAME|V2DOCUMENTIDENTIFIER|            V1THEMES|         V1LOCATIONS|           V1PERSONS|              V1TONE|
+-----------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 20200101000000-0|20200101000000|    savingadvice.com|https://wink.savi...|USPEC_POLICY1;EPU...|                    |                    |2.57731958762887,...|
| 20200101000000-1|20200101000000|      torontosun.com|https://torontosu...|TAX_FNCACT;TAX_FN...|1#Canada#

In [6]:
val mentionRDD = sc.binaryFiles("s3://testfuret/Master_file /*mentions*",100).    
   flatMap {  
       
       // decompresser les fichiers
       case (name: String, content: PortableDataStream) =>

          val zis = new ZipInputStream(content.open)
          Stream.continually(zis.getNextEntry).
                takeWhile(_ != null).
                flatMap { _ =>
                    val br = new BufferedReader(new InputStreamReader(zis))
                    Stream.continually(br.readLine()).takeWhile(_ != null)
                }

    }

val mentionDF = mentionRDD.map(x => x.split("\t")).map(row => row.mkString(";")).map(x => x.split(";")).toDF()



val dfMention = mentionDF.withColumn("_tmp", $"value").select(
     $"_tmp".getItem(0).as("globaleventID"),
     //$"_tmp".getItem(1).as("EventTimeDate"),
     //$"_tmp".getItem(2).as("MentionTimeDate"),
     //$"_tmp".getItem(3).as("MentionType"),
     $"_tmp".getItem(4).as("MentionSourceName"),
     $"_tmp".getItem(5).as("MentionIdentifier"),
     //$"_tmp".getItem(6).as("SentenceID"),
     //$"_tmp".getItem(7).as("Actor1CharOffset"),
     //$"_tmp".getItem(8).as("Actor2CharOffset"),
     //$"_tmp".getItem(9).as("ActionCharOffset"),
     //$"_tmp".getItem(10).as("InRawText"),
     //$"_tmp".getItem(11).as("Confidence"),
     //$"_tmp".getItem(12).as("MentionDocLen"),
     //$"_tmp".getItem(13).as("MentionDocTone"),
     $"_tmp".getItem(14).as("MentionDocTranslationInfo")
     //$"_tmp".getItem(15).as("Extras")
     )

dfMention.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

mentionRDD: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[22] at flatMap at <console>:56
mentionDF: org.apache.spark.sql.DataFrame = [value: array<string>]
dfMention: org.apache.spark.sql.DataFrame = [globaleventID: string, MentionSourceName: string ... 2 more fields]
+-------------+--------------------+--------------------+-------------------------+
|globaleventID|   MentionSourceName|   MentionIdentifier|MentionDocTranslationInfo|
+-------------+--------------------+--------------------+-------------------------+
|    813459872|          whas11.com|https://www.whas1...|                     null|
|    896047517|          whas11.com|https://www.whas1...|                     null|
|    813505048|          whas11.com|https://www.whas1...|                     null|
|    896047518|         recorder.ca|https://www.recor...|                     null|
|    896047519|          smh.com.au|https://www.smh.c...|                     null|
|    896047520|     dailymail.co.uk|https://www.daily

In [7]:
val EventRDD = sc.binaryFiles("s3://testfuret/Master_file /*export*",100).    
   flatMap {  
       
       // decompresser les fichiers
       case (name: String, content: PortableDataStream) =>

          val zis = new ZipInputStream(content.open)
          Stream.continually(zis.getNextEntry).
             takeWhile(_ != null).
             flatMap { _ =>
                val br = new BufferedReader(new InputStreamReader(zis))
                Stream.continually(br.readLine()).takeWhile(_ != null)
                }
         
        
           

    }





val eventDF = EventRDD.map(x => x.split("\t")).map(row => row.mkString(";")).map(x => x.split(";")).toDF()


val dfEvent = eventDF.withColumn("_tmp", $"value").select(
     $"_tmp".getItem(0).as("globaleventID"),
     $"_tmp".getItem(1).as("day"),
     $"_tmp".getItem(2).as("month"),
     $"_tmp".getItem(3).as("year"),
     //$"_tmp".getItem(4).as("fractiondate"),
     //$"_tmp".getItem(5).as("Actor1code"),
     //$"_tmp".getItem(6).as("Actor1Name"),
     //$"_tmp".getItem(7).as("Actor1ContryCode"),
     //$"_tmp".getItem(8).as("actor1KnownGroupeCode"),
     //$"_tmp".getItem(9).as("Actor1EthnicCode"),
     //$"_tmp".getItem(10).as("Actor1Religion1Code"),
     //$"_tmp".getItem(11).as("Actor1Religion2code"),
     //$"_tmp".getItem(12).as("Actor1type1Code"),
     //$"_tmp".getItem(13).as("Actor1Type2code"),
     //$"_tmp".getItem(14).as("Actor1Type3code"),
     //$"_tmp".getItem(15).as("Actor2code"),
     //$"_tmp".getItem(16).as("Actor2Name"),
     //$"_tmp".getItem(17).as("Actor2ContryCode"),
     //$"_tmp".getItem(18).as("Actor2KnownGroupeCode"),
     //$"_tmp".getItem(19).as("Actor2EthnicCode"),
     //$"_tmp".getItem(20).as("Actor2Religion1Code"),
     //$"_tmp".getItem(21).as("Actor2Religion2code"),
     //$"_tmp".getItem(22).as("Actor2type1Code"),
     //$"_tmp".getItem(23).as("Actor2Type2code"),
     //$"_tmp".getItem(24).as("Actor2Type3code"),
     //$"_tmp".getItem(25).as("IsRootEvent"),
     //$"_tmp".getItem(26).as("EventCode"),
     //$"_tmp".getItem(27).as("EventBaseCode"),
     //$"_tmp".getItem(28).as("EventRootCode"),
     //$"_tmp".getItem(29).as("QuadClass"),
     //$"_tmp".getItem(30).as("GoldsteinScale"),
     $"_tmp".getItem(31).as("NumMentions"),
     $"_tmp".getItem(32).as("NumSources"),
     $"_tmp".getItem(33).as("NumArticles"),
     $"_tmp".getItem(34).as("AvgTone"),
     //$"_tmp".getItem(35).as("Actor1Geo_Type"),
     //$"_tmp".getItem(36).as("Actor1Geo_Fullname"),
     //$"_tmp".getItem(37).as("Actor1Geo_CountryCode"),
     //$"_tmp".getItem(38).as("Actor1Geo_ADM1Code"),
     //$"_tmp".getItem(39).as("Actor1Geo_ADM2Code"),
     //$"_tmp".getItem(40).as("Actor1Geo_Lat"),
     //$"_tmp".getItem(41).as("Actor1Geo_Long"),
     //$"_tmp".getItem(42).as("Actor1Geo_FeatureID"),
     //$"_tmp".getItem(43).as("Actor2Geo_Type"),
     //$"_tmp".getItem(44).as("Actor2Geo_Fullname"),
     //$"_tmp".getItem(45).as("Actor2Geo_CountryCode"),
     //$"_tmp".getItem(46).as("Actor2Geo_ADM1Code"),
     //$"_tmp".getItem(47).as("Actor2Geo_ADM2Code"),
     //$"_tmp".getItem(48).as("Actor2Geo_Lat"),
     //$"_tmp".getItem(49).as("Actor2Geo_Long"),
     //$"_tmp".getItem(50).as("Actor2Geo_FeatureID"),
     //$"_tmp".getItem(51).as("ActionGeo_Type"),
     $"_tmp".getItem(52).as("ActionGeo_Fullname")
     //$"_tmp".getItem(53).as("ActionGeo_CountryCode"),
     //$"_tmp".getItem(54).as("ActionGeo_ADM1Code"),
     //$"_tmp".getItem(55).as("ActionGeo_ADM2Code"),
     //$"_tmp".getItem(56).as("ActionGeo_Lat"),
     //$"_tmp".getItem(57).as("ActionGeo_Long"),
     //$"_tmp".getItem(58).as("ActionGeo_FeatureID"),
     //$"_tmp".getItem(59).as("DATEADDED")
    )


dfEvent.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

EventRDD: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[31] at flatMap at <console>:56
eventDF: org.apache.spark.sql.DataFrame = [value: array<string>]
dfEvent: org.apache.spark.sql.DataFrame = [globaleventID: string, day: string ... 7 more fields]
+-------------+--------+------+----+-----------+----------+-----------+------------------+--------------------+
|globaleventID|     day| month|year|NumMentions|NumSources|NumArticles|           AvgTone|  ActionGeo_Fullname|
+-------------+--------+------+----+-----------+----------+-----------+------------------+--------------------+
|    896047517|20190101|201901|2019|          2|         1|          2|                 0|Kentucky, United ...|
|    896047518|20190101|201901|2019|          1|         1|          1|  -0.7164790174002|Thousand Islands,...|
|    896047519|20190101|201901|2019|          5|         1|          5| -0.95238095238095|Sydney, New South...|
|    896047520|20190101|201901|2019|          8|         1|          8| -

# Filtre Coronavirus du la datframe GKG

### "V1THEMES In lowercasr 

In [8]:
// on met toute la colonne V1THEMES en minuscule
val GKG2_low = dfGKG.withColumn("V1THEMES",lower($"V1THEMES")) 

val GKG2_low_coro = GKG2_low.filter($"V1THEMES".like("%coronavirus%"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

GKG2_low: org.apache.spark.sql.DataFrame = [GKGRECORDID: string, V2DATE: string ... 6 more fields]
GKG2_low_coro: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [GKGRECORDID: string, V2DATE: string ... 6 more fields]


In [9]:
GKG2_low_coro.select("V1THEMES").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|            V1THEMES|
+--------------------+
|tax_ethnicity;tax...|
|general_health;me...|
|tax_disease;tax_d...|
|general_health;me...|
|tourism;tax_fncac...|
|soc_quarantine;ta...|
|soc_quarantine;ta...|
|soc_quarantine;ta...|
|soc_quarantine;ta...|
|soc_quarantine;ta...|
|soc_quarantine;ta...|
|soc_quarantine;ta...|
|general_health;me...|
|soc_quarantine;ta...|
|soc_quarantine;ta...|
|soc_quarantine;ta...|
|soc_quarantine;ta...|
|soc_quarantine;ta...|
|soc_quarantine;ta...|
|soc_quarantine;ta...|
+--------------------+
only showing top 20 rows



# Join  dfEvent dfGKG dfMention

In [10]:
val dfGKG_Pand = dfGKG.
        withColumn("CheckCOVID", $"V1THEMES".like("%PANDEMIC%"))
dfGKG_Pand.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dfGKG_Pand: org.apache.spark.sql.DataFrame = [GKGRECORDID: string, V2DATE: string ... 7 more fields]
+-----------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|      GKGRECORDID|        V2DATE|  V2SOURCECOMMONNAME|V2DOCUMENTIDENTIFIER|            V1THEMES|         V1LOCATIONS|           V1PERSONS|              V1TONE|CheckCOVID|
+-----------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
| 20200101000000-0|20200101000000|    savingadvice.com|https://wink.savi...|USPEC_POLICY1;EPU...|                    |                    |2.57731958762887,...|     false|
| 20200101000000-1|20200101000000|      torontosun.com|https://torontosu...|TAX_FNCACT;TAX_FN...|1#Canada#CA#CA#60...|david lepofsky;do...|-2.10896309314587...|     false|
| 20200101000000-2|20200101000000|     

In [11]:
val dfGKG_Pandemic = dfGKG_Pand.
    filter((dfGKG_Pand("CheckCOVID") === "true"))
dfGKG_Pandemic.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dfGKG_Pandemic: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [GKGRECORDID: string, V2DATE: string ... 7 more fields]
+------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|       GKGRECORDID|        V2DATE|  V2SOURCECOMMONNAME|V2DOCUMENTIDENTIFIER|            V1THEMES|         V1LOCATIONS|           V1PERSONS|              V1TONE|CheckCOVID|
+------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
| 20200101000000-25|20200101000000|           axios.com|https://www.axios...|TAX_FNCACT;TAX_FN...|1#China#CH#CH#35#...|alibaba xianyu;yi...|-5.02283105022831...|      true|
| 20200101000000-44|20200101000000|    bendbulletin.com|https://www.bendb...|TAX_FNCACT;TAX_FN...|3#Siskiyou Mounta...|lonnie seiders;ne...|-1.07744107744108...|      true|
| 2020

In [12]:
val dfGKG_CO = dfGKG.
    withColumn("CheckCOVID", $"V1THEMES".like("%CORONAVIRUS%"))
dfGKG_CO.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dfGKG_CO: org.apache.spark.sql.DataFrame = [GKGRECORDID: string, V2DATE: string ... 7 more fields]
+-----------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|      GKGRECORDID|        V2DATE|  V2SOURCECOMMONNAME|V2DOCUMENTIDENTIFIER|            V1THEMES|         V1LOCATIONS|           V1PERSONS|              V1TONE|CheckCOVID|
+-----------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
| 20200101000000-0|20200101000000|    savingadvice.com|https://wink.savi...|USPEC_POLICY1;EPU...|                    |                    |2.57731958762887,...|     false|
| 20200101000000-1|20200101000000|      torontosun.com|https://torontosu...|TAX_FNCACT;TAX_FN...|1#Canada#CA#CA#60...|david lepofsky;do...|-2.10896309314587...|     false|
| 20200101000000-2|20200101000000|       

In [13]:
val dfGKG_Corona = dfGKG_CO.
    filter((dfGKG_CO("CheckCOVID") === "true"))
dfGKG_Corona.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dfGKG_Corona: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [GKGRECORDID: string, V2DATE: string ... 7 more fields]
+-------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|        GKGRECORDID|        V2DATE|  V2SOURCECOMMONNAME|V2DOCUMENTIDENTIFIER|            V1THEMES|         V1LOCATIONS|           V1PERSONS|              V1TONE|CheckCOVID|
+-------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
| 20200101000000-664|20200101000000|             umn.edu|http://www.cidrap...|TAX_ETHNICITY;TAX...|1#Taiwan#TW#TW#24...|            leo poon|-4.02298850574713...|      true|
| 20200101021500-680|20200101021500|           3seer.net|https://www.3seer...|GENERAL_HEALTH;ME...|1#Saudi Arabia#SA...|                    |-1.72413793103448...|      true|
|20

In [14]:
val dfGKG_Covid = dfGKG_Pandemic.union(dfGKG_Corona).distinct()
//dfGKG_Covid.show()
//val dfGKG_Covid=dfGKG_Pandemic.union( broadcast(dfGKG_Corona)).distinct()
//val brod_fGKG_Covid=broadcast(dfGKG_Covid)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dfGKG_Covid: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [GKGRECORDID: string, V2DATE: string ... 7 more fields]


In [25]:
//dfGKG_Covid.write.mode(SaveMode.Overwrite).format("parquet").saveAsTable("s3://testfuret/result/dfGKG_Corona.parquet")
//dfGKG_Covid.write.mode.save("s3://testfuret/result/dfGKG_Corona.parquet", SaveMode.Append )
dfGKG_Covid.write.format("parquet").mode("append").save("dfGKG_Covid.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

KeyboardInterrupt: 

In [12]:
import org.apache.spark.sql.functions.{concat_ws, col}
import org.apache.spark.sql.Row

val expr = concat_ws(",", dfGKG_Covid.columns.map(col): _*)
//df.select(expr).map(_.getString(0)).saveAsTextFile("some_path")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.sql.functions.{concat_ws, col}
import org.apache.spark.sql.Row
expr: org.apache.spark.sql.Column = concat_ws(,, GKGRECORDID, V2DATE, V2SOURCECOMMONNAME, V2DOCUMENTIDENTIFIER, V1THEMES, V1LOCATIONS, V1PERSONS, V1TONE, CheckCOVID)


In [13]:
dfGKG_Covid.select(expr).map(_.getString(0)).rdd.saveAsTextFile("s3://testfuret/result/dfGKG_Corona.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Invalid status code '404' from http://localhost:8998/sessions/0/statements/13 with error payload: {"msg":"Session '0' not found."}


In [10]:
dfGKG_Covid.write.csv("s3://testfuret/result/dfGKG_Corona.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Invalid status code '404' from http://localhost:8998/sessions/1/statements/10 with error payload: {"msg":"Session '1' not found."}


In [8]:

val joinedDF = dfMention.join(dfEvent, dfMention("globaleventID")===dfEvent("globaleventID"), "inner").  //prob
	drop(dfEvent("globaleventID"))


    val dfGKG_Pand = dfGKG.
        withColumn("CheckCOVID", $"V1THEMES".like("%PANDEMIC%"))

val dfGKG_Pandemic = dfGKG_Pand.
    filter((dfGKG_Pand("CheckCOVID") === "true"))

val dfGKG_CO = dfGKG.
    withColumn("CheckCOVID", $"V1THEMES".like("%CORONAVIRUS%"))

//refaire la même avec le thème "COVID-19", "COVID19"

val dfGKG_Corona = dfGKG_CO.
    filter((dfGKG_CO("CheckCOVID") === "true"))

val dfGKG_Covid = dfGKG_Pandemic.union(dfGKG_Corona).distinct()


val dfGKGCovidWithMentionEvent = dfGKG_Covid.join(joinedDF, dfGKG_Covid("V2DOCUMENTIDENTIFIER")===joinedDF("MentionIdentifier"), "inner")

val dfCovid_Pays = dfGKGCovidWithMentionEvent.
	withColumn("Pays", split($"ActionGeo_Fullname", ", ")).
    withColumn("Pays", element_at($"Pays", -1))
    

//TRANSFORMATION EN JSON
//dfCovid_Pays.write.json("/Users/amandineguignard/Documents/ROLLY/Telecom/CoursTelecom/BDD_non_relationnelles/Projet/JsonDF")
//dfCovid_Pays.coalesce(1).write.json("/Users/amandineguignard/Documents/ROLLY/Telecom/CoursTelecom/BDD_non_relationnelles/Projet/Json1File")

//TRANSFORMATION EN CSV
//dfGKGCovidWithMentionEvent.write.csv("/Users/amandineguignard/Documents/ROLLY/Telecom/CoursTelecom/BDD_non_relationnelles/Projet/csvDF")

//TRANSFORMATION EN PARQUET
//dfGKGCovidWithMentionEvent.write.parquet("/Users/amandineguignard/Documents/ROLLY/Telecom/CoursTelecom/BDD_non_relationnelles/Projet/Parquet")




VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

joinedDF: org.apache.spark.sql.DataFrame = [globaleventID: string, MentionSourceName: string ... 10 more fields]
dfGKG_Pand: org.apache.spark.sql.DataFrame = [GKGRECORDID: string, V2DATE: string ... 7 more fields]
dfGKG_Pandemic: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [GKGRECORDID: string, V2DATE: string ... 7 more fields]
dfGKG_CO: org.apache.spark.sql.DataFrame = [GKGRECORDID: string, V2DATE: string ... 7 more fields]
dfGKG_Corona: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [GKGRECORDID: string, V2DATE: string ... 7 more fields]
dfGKG_Covid: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [GKGRECORDID: string, V2DATE: string ... 7 more fields]
dfGKGCovidWithMentionEvent: org.apache.spark.sql.DataFrame = [GKGRECORDID: string, V2DATE: string ... 19 more fields]
dfCovid_Pays: org.apache.spark.sql.DataFrame = [GKGRECORDID: string, V2DATE: string ... 20 more fields]


In [9]:
// 1°) afficher le nombre d’articles/évènements qui parlent de COVID qu’il y a eu pour chaque triplet (jour, pays de l’évènement, langue de l’article).

def cleanLangue(langue: String): String = {
	if (langue == null)
    	"ENG" 
	else
        langue.slice(langue.length -7,langue.length -4)
}

val cleanLangueUdf = udf(cleanLangue _)

val dfLangue= dfCovid_Pays.
	withColumn("Langue", cleanLangueUdf($"MentionDocTranslationInfo")).
    drop($"MentionDocTranslationInfo")

val CountEventArticle = dfLangue.
	groupBy("day", "Pays", "Langue").
    agg(countDistinct("globaleventID").alias("NbEvent"),countDistinct("GKGRECORDID").alias("NbArticle"))

//Requête Question 1)
//CountEventArticle.show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

cleanLangue: (langue: String)String
cleanLangueUdf: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function1>,StringType,Some(List(StringType)))
dfLangue: org.apache.spark.sql.DataFrame = [GKGRECORDID: string, V2DATE: string ... 20 more fields]
CountEventArticle: org.apache.spark.sql.DataFrame = [day: string, Pays: string ... 3 more fields]


In [10]:
CountEventArticle.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
org.apache.spark.SparkException: Job aborted due to stage failure: Task 22 in stage 6.0 failed 4 times, most recent failure: Lost task 22.3 in stage 6.0 (TID 32, ip-172-31-45-251.ec2.internal, executor 3): java.io.EOFException: Unexpected end of ZLIB input stream
	at java.util.zip.InflaterInputStream.fill(InflaterInputStream.java:240)
	at java.util.zip.InflaterInputStream.read(InflaterInputStream.java:158)
	at java.util.zip.ZipInputStream.read(ZipInputStream.java:194)
	at sun.nio.cs.StreamDecoder.readBytes(StreamDecoder.java:284)
	at sun.nio.cs.StreamDecoder.implRead(StreamDecoder.java:326)
	at sun.nio.cs.StreamDecoder.read(StreamDecoder.java:178)
	at java.io.InputStreamReader.read(InputStreamReader.java:184)
	at java.io.BufferedReader.fill(BufferedReader.java:161)
	at java.io.BufferedReader.readLine(BufferedReader.java:324)
	at java.io.BufferedReader.readLine(BufferedReader.java:389)
	at $anonfun$1$$anonfun$apply$3$$anonfun$apply$4.apply(<console>:66)
	at $an